##  导入依赖库
1. pandas，用于读取xlsx表格
2. base64，用于编码/解码算例文件
3. json，用于编码/解码json字符串
4. redis，用于服务器与客户端的通信
5. threading，用于发布/订阅的并发

In [1]:
import pandas as pd
import base64,json
import redis,threading
host = "localhost"

## 优化计算结果频道订阅
在接收到服务器的运行结果后，将其储存为excel文件

In [2]:
def read_response():
    r = redis.Redis(host=host,port=6379)
    print("opt_response channel subscribed")
    sub = r.pubsub()
    sub.subscribe("opt_response")
    resp = sub.parse_response()
    while 1:
        resp = sub.parse_response()
        resp = resp[2]
        print("message received")
        d = json.loads(resp)
        print(d["case_info"])
        for filename in ["agc","nonagc","wind"]:
            encoded_file = d[filename]
            filepath = d["ID"] + filename
            with open("../results/"+filepath+".csv","wb") as f:
                csv_file = base64.b64decode(encoded_file)
                f.write(csv_file)
                print("result saved as {0}".format(filepath))

## 建立redis连接

In [3]:
r = redis.Redis(host=host,port=6379)
print("connection established")

connection established


## 开始优化计算结果频道订阅

In [4]:
p = threading.Thread(target=read_response)
p.start()

opt_response channel subscribed


##  发布计算请求
1. 读取负荷预测、可再生能源预测Excel表
2. 读取算例文件
3. 结合用户参数配置，生成json消息
4. 发布到计算请求频道

In [10]:
load = pd.read_csv("../input/load data new.csv")
casefile = open("../input/pglib_opf_case1354_pegase.m",'rb')
# case_encoded = base64.b64encode(casefile.read())
case_encoded = casefile.read()
case_encoded_json = str(case_encoded, encoding = "utf-8")
for confidence in [98]:
    for k in [1]:#多目标权重循环
        df = pd.read_csv("../input/" + str(confidence) + "wind - 96.csv")
        WIND = [dict(bus=1,upper_bound=df["max"].tolist(),lower_bound=df["min"].tolist())]
        system_load = load["load96"].tolist()
        d = {"ID":str(confidence)+'_'+str(k),
            "WIND":WIND,
            "system_load":system_load,
            "case":case_encoded_json,
            "confidence":confidence,
            "windCur_weight":k,
            "Method":-1}
        json_msg = json.dumps(d)
        r.publish("opt_requests",json_msg)
        print('message sent')

message sent
message received
{'windPercentMax': 0.3, 'nbus': 1354, 'totalLoadMin': 658.4218121720278, 'ngen': 260, 'totalPmax': 1287.3860000000004, 'totalLoadMax': 1158.6474000000005, 'nbranch': 1991, 'totalWindMin': 75.21646740854288, 'totalWindMax': 230.91428166673344}
result saved as 98_1agc
result saved as 98_1nonagc
result saved as 98_1wind
message received
{'windPercentMax': 0.3, 'nbus': 300, 'totalLoadMin': 213.4095569095416, 'ngen': 69, 'totalPmax': 326.78435, 'totalLoadMax': 294.10591500000004, 'nbranch': 411, 'totalWindMin': 19.092614345190075, 'totalWindMax': 58.6142566721872}
result saved as 90_1agc
result saved as 90_1nonagc
result saved as 90_1wind
message received
{'windPercentMax': 0.3, 'nbus': 300, 'totalLoadMin': 213.4095569095416, 'ngen': 69, 'totalPmax': 326.78435, 'totalLoadMax': 294.10591500000004, 'nbranch': 411, 'totalWindMin': 19.092614345190075, 'totalWindMax': 58.6142566721872}
result saved as 90_1agc
result saved as 90_1nonagc
result saved as 90_1wind
messa

message received
{'windPercentMax': 0.3, 'nbus': 300, 'totalLoadMin': 199.4950923774687, 'ngen': 69, 'totalPmax': 326.78435, 'totalLoadMax': 294.10591500000004, 'nbranch': 411, 'totalWindMin': 19.092614345190075, 'totalWindMax': 58.61425667218721}
result saved as 90_1agc
result saved as 90_1nonagc
result saved as 90_1wind
message received
{'windPercentMax': 0.3, 'nbus': 300, 'totalLoadMin': 201.95895692115894, 'ngen': 69, 'totalPmax': 326.78435, 'totalLoadMax': 294.10591500000004, 'nbranch': 411, 'totalWindMin': 19.092614345190075, 'totalWindMax': 58.61425667218721}
result saved as 90_1agc
result saved as 90_1nonagc
result saved as 90_1wind
message received
{'windPercentMax': 0.3, 'nbus': 300, 'totalLoadMin': 209.7022007051843, 'ngen': 69, 'totalPmax': 326.78435, 'totalLoadMax': 294.10591500000004, 'nbranch': 411, 'totalWindMin': 19.092614345190075, 'totalWindMax': 58.61425667218721}
result saved as 90_1agc
result saved as 90_1nonagc
result saved as 90_1wind
message received
{'windPerc

### with on_off var reduction
Best objective 9.527461382980e+06, best bound 9.526644262663e+06, gap 0.0086%, cpu time 38.52 seconds

### without on_off var reduction
Best objective 9.510280348464e+06, best bound 9.509412630681e+06, gap 0.0091%, cpu time 86.68 seconds

import pandas as pd
from scipy.interpolate import interp1d

df = pd.read_csv("../input/load data.csv")
df.head()
temp = df['load'].tolist()[0:24]
f = interp1d(range(24), temp,kind='cubic')

load_new = []
for t in range(96):
    y = f(23*t/96)
    load_new.append(float(y))
load_new = pd.Series(load_new)
load_new.plot()
# load_new.diff().plot()
df["load96"] = load_new
df.to_csv("../input/load data new.csv")

{
    "confidence":0.9,#置信度
    "objective_weights":{
        "economical":0.5,#经济性
        "environmental":0.25,#环保性
        "renewable":0.25#新能源消纳
    },#多目标优化权重
    "casefile":"case300.m",#matpower格式的算例文件
    "forecast":{
        "renewable":
            [{"bus":1,#新能源电站所处母线编号
                "upperbound":list(),#预测出力上界
                "lowerbound":list()#预测出力下界
            }],
        "load":[{"bus":1,#负荷所处母线编号
                "upperbound":list(),#预测出力上界
                "lowerbound":list()#预测出力下界
            }]
    },
    "multi_objective":1,#多目标优化模式，1代表多目标，0代表单目标
    "fast_mode":0,#快速计算模式，1代表快速计算，0代表精确计算
    "MIP_Gap":0.0001,#求解器对偶间隙
    "algorithm":-1,#求解器算法，-1代表默认值
}

{
    "system_info":{
        'windPercentMax': 0.3,#最大风电占比
        'nbus': 300, #节点数
        'totalLoadMin': 167,#总负荷最小值
        'ngen': 69, #机组数
        'totalPmax': 326.78,#机组总容量
        'totalLoadMax': 294,#总负荷最大值
        'nbranch': 411,#线路数
        'totalWindMin': 19,#总风电出力最小值
        'totalWindMax': 58.6#总风电出力最大值
    },#系统概要
    "results":[
        {
            "objective_weights":{
                "economical":0.5,#经济性
                "environmental":0.25,#环保性
                "renewable":0.25#新能源消纳
            },#多目标优化权重
            "fixed":'fixed_0.xlsx',#不含运行区间的机组调度计划excel文件名，包含启停计划、基准出力
            "variable":'variable_0.xlsx'#含运行区间的机组调度计划excel文件名，包含启停计划、基准出力、出力区间
            "objective_value":{
                "economical":10124523,#运行成本
                "environmental":324,#二氧化碳排放量
                "renewable":0.95#新能源消纳率
            },#目标函数最优值
        },
        {
            "objective_weights":{
                "economical":0.25,#经济性
                "environmental":0.5,#环保性
                "renewable":0.25#新能源消纳
            },#多目标优化权重
            "fixed":'fixed_1.xlsx',#不含运行区间的机组调度计划excel文件名，包含启停计划、基准出力
            "variable":'variable_1.xlsx'#含运行区间的机组调度计划excel文件名，包含启停计划、基准出力、出力区间
            "objective_value":{
                "economical":10524523,#运行成本
                "environmental":300,#二氧化碳排放量
                "renewable":0.95#新能源消纳率
            },#目标函数最优值
        }
    ]
}